<a href="https://colab.research.google.com/github/qamtam/ads_final_project/blob/main/Projekt_Abel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#let's try to make our very own embeddings of Polish language
import tensorflow as tf
import pandas as pd
import numpy as np
import io
import itertools
import os
import re
import string
import tensorflow as tf
import tqdm
import requests
import urllib.request
import tarfile
import shutil
import csv
from bs4 import BeautifulSoup
from os import walk
from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import Activation, Dense, Dot, Embedding, Flatten, GlobalAveragePooling1D, Reshape
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization


In [ ]:
#preprocess raw data and get pandas-friendly csv
cwd = os.getcwd()
data_path = os.path.join(cwd, 'Data')
dest = os.path.join(data_path, 'dest')

In [ ]:
url = 'http://clip.ipipan.waw.pl/NationalCorpusOfPolish?action=AttachFile&do=get&target=NKJP-PodkorpusMilionowy-1.2.tar.gz'
#^ this is the hand-annotated fragment of the full corpus, couple hundred MBs
thetarfile = url
ftpstream = urllib.request.urlopen(thetarfile)
thetarfile = tarfile.open(fileobj=ftpstream, mode="r|gz")
thetarfile.extractall(data_path)
# this should unpack the archive in /Data folder

In [ ]:
try:
  os.mkdir(dest) 
except FileExistsError as err:
  pass

i = 0
# extract only pure text files (text.xml files), with not too much metadata
for subdir, dirs, files in os.walk(data_path):
  for file in files:
      if file == "text.xml":
          i+=1
          base, extension = os.path.splitext(file)
          destination = os.path.join(dest, '{}_{}{}'.format(base, i, extension))
          original = os.path.join(subdir, file)
          shutil.copyfile(original, destination)
          #^ copy files one by one to /Data/dest

In [ ]:
nawlg = pd.read_excel('https://static-content.springer.com/esm/art%3A10.3758%2Fs13428-014-0552-1/MediaObjects/13428_2014_552_MOESM1_ESM.xlsx')


url = 'https://drive.google.com/file/d/1frgNyuSUZ9qQKma3CskKof7pLFGX0C7v/view?usp=sharing' 
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
nawl = pd.read_csv(path)
#^ path to hand-downloaded file with sentiment/emotion analysis of about a thousand words 
# more info : https://exp.lobi.nencki.gov.pl/nawl-analysis
nawlg = nawlg[["NAWL_word", "Gram"]]


names = pd.read_csv('https://raw.githubusercontent.com/sdadas/polish-nlp-resources/master/lexicons/names/names.csv')
names
names.columns = ['name', 'metadata', 'u']
is_name =  names['metadata']=='P-N' #leaves only first Polish names
first_names = names[is_name].name


#NAWL researches sentiments behind popular words in basic emotions
# Happiness, Anger, Sadness, Fear, Disgust
# I'll simplify the dataset and think of 'happy' words as positive and every other type as negative


nawlg.columns = ['word', 'gram']

nawl.sort_values(by='word', ascending=True)


words_df = pd.merge(nawl, nawlg, on=['word']) # main nawl table doesn't have grammar, nawlg doesn't define categories as they are left up to the user

positive_adjectives = words_df[(words_df['category'] == 'H') & (words_df['gram'] == 3)].word #gram == 3 is adjective
negative_adjectives = words_df[(words_df['category'].isin(['A', 'S', 'D', 'F'])) & (words_df['gram'] == 3)].word
positive_words = words_df[(words_df['category'] == 'H')].word
negative_words = words_df[(words_df['category'].isin(['A', 'S', 'D', 'F']))].word
# handy shortcut of ASDF - anger sadness disgust and fear


popular_names  = pd.read_csv('https://api.dane.gov.pl/resources/21457,imiona-nadane-dzieciom-w-polsce-w-latach-2000-2019-imie-pierwsze/file') # finding this link was a PITA
#names given to Polish babies in 21st century; sorted by popularity in any given year
summed_names = popular_names.groupby(['Imię', 'Płeć']).agg({'Liczba': 'sum'}).sort_values(by= 'Liczba') # wanting to preserve information about sex creates a MultiIndex
girl_names = summed_names[summed_names.index.isin(['K'], level=1)]
boy_names = summed_names[summed_names.index.isin(['M'], level=1)]

def get_names_list(names_table, top_n):
  names_table.reset_index(inplace=True)
  names_table.drop(columns='Płeć', inplace=True)
  new = names_table.sort_values(by='Liczba', ascending=False)[:top_n]
  new.drop(columns='Liczba', inplace=True)
  new = pd.Series(new['Imię'])
  l = []
  for name in new:
    l.append(name.lower().capitalize())
  return pd.Series(l)

pop_boy_names = get_names_list(boy_names, 170)
pop_girl_names = get_names_list(girl_names, 170)
pop_names = get_names_list(summed_names, 400)
all_names = pd.concat((pop_names,first_names), copy=False)
all_names.drop_duplicates(inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
_, _, filenames = next(walk(dest))
print(filenames)
outputcsv = os.path.join(dest, "output.csv")
#copy all of pure text content in the corpus to a single csv file
with open(outputcsv, 'w', newline='', encoding="utf-8") as csvfile:
    writer = csv.writer(csvfile)
    i = 0
    for file in filenames:
      i += 1
      print("##################")
      print(i)
      print("##################")
      fi = os.path.join(dest, file)
      with open(fi, "r", encoding="utf-8") as f:
        contents = f.read()
        soup = BeautifulSoup(contents, 'lxml')
        texts = soup.find_all("ab")
        writer.writerows(texts)

Streaming output truncated to the last 5000 lines.
2223
##################
##################
2224
##################
##################
2225
##################
##################
2226
##################
##################
2227
##################
##################
2228
##################
##################
2229
##################
##################
2230
##################
##################
2231
##################
##################
2232
##################
##################
2233
##################
##################
2234
##################
##################
2235
##################
##################
2236
##################
##################
2237
##################
##################
2238
##################
##################
2239
##################
##################
2240
##################
##################
2241
##################
##################
2242
##################
##################
2243
##################
##################
2244
##################
##################
224

In [ ]:
data = pd.read_csv(outputcsv, header=None)
pd.set_option('display.max_colwidth', None)

In [ ]:
data.head(20)

,0
0,– tak..
1,– którą też przeczytałam całą. tylko mi się rozkleiła na kawałki. więc kupiłam ci drugą..
2,– no nie żartuj..
3,– było mi głupio wiesz co..
4,– e tam..
5,– słuchaj kartka po kartce naprawdę ja już w ogóle wiesz. i ona jeździła ze mną wszędzie więc mówię kurczę aż mi głupio taką oddać nie. mówię tutaj wiesz chyba że chcesz chlebka razowego..
6,– nie dziękuję..
7,– masz zdjątka jakieś fajne widzę..
8,– no byłaś bo druga wycieczka była po tej samej trasie..
9,– może..


In [ ]:
#simplify the analyzed texts

def removeAccents(input_text):
    strange='ŮôῡΒძěἊἦëĐᾇόἶἧзвŅῑἼźἓŉἐÿἈΌἢὶЁϋυŕŽŎŃğûλВὦėἜŤŨîᾪĝžἙâᾣÚκὔჯᾏᾢĠфĞὝŲŊŁČῐЙῤŌὭŏყἀхῦЧĎὍОуνἱῺèᾒῘᾘὨШūლἚύсÁóĒἍŷöὄЗὤἥბĔõὅῥŋБщἝξĢюᾫაπჟῸდΓÕűřἅгἰშΨńģὌΥÒᾬÏἴქὀῖὣᾙῶŠὟὁἵÖἕΕῨčᾈķЭτἻůᾕἫжΩᾶŇᾁἣჩαἄἹΖеУŹἃἠᾞåᾄГΠКíōĪὮϊὂᾱიżŦИὙἮὖÛĮἳφᾖἋΎΰῩŚἷРῈĲἁéὃσňİΙῠΚĸὛΪᾝᾯψÄᾭêὠÀღЫĩĈμΆᾌἨÑἑïოĵÃŒŸζჭᾼőΣŻçųøΤΑËņĭῙŘАдὗპŰἤცᾓήἯΐÎეὊὼΘЖᾜὢĚἩħĂыῳὧďТΗἺĬὰὡὬὫÇЩᾧñῢĻᾅÆßшδòÂчῌᾃΉᾑΦÍīМƒÜἒĴἿťᾴĶÊΊȘῃΟúχΔὋŴćŔῴῆЦЮΝΛῪŢὯнῬũãáἽĕᾗნᾳἆᾥйᾡὒსᾎĆрĀüСὕÅýფᾺῲšŵкἎἇὑЛვёἂΏθĘэᾋΧĉᾐĤὐὴιăąäὺÈФĺῇἘſგŜæῼῄĊἏØÉПяწДĿᾮἭĜХῂᾦωთĦлðὩზკίᾂᾆἪпἸиᾠώᾀŪāоÙἉἾρаđἌΞļÔβĖÝᾔĨНŀęᾤÓцЕĽŞὈÞუтΈέıàᾍἛśìŶŬȚĳῧῊᾟάεŖᾨᾉςΡმᾊᾸįᾚὥηᾛġÐὓłγľмþᾹἲἔбċῗჰხοἬŗŐἡὲῷῚΫŭᾩὸùᾷĹēრЯĄὉὪῒᾲΜᾰÌœĥტ'
    ascii_replacements='UoyBdeAieDaoiiZVNiIzeneyAOiiEyyrZONgulVoeETUiOgzEaoUkyjAoGFGYUNLCiIrOOoqaKyCDOOUniOeiIIOSulEySAoEAyooZoibEoornBSEkGYOapzOdGOuraGisPngOYOOIikoioIoSYoiOeEYcAkEtIuiIZOaNaicaaIZEUZaiIaaGPKioIOioaizTIYIyUIifiAYyYSiREIaeosnIIyKkYIIOpAOeoAgYiCmAAINeiojAOYzcAoSZcuoTAEniIRADypUitiiIiIeOoTZIoEIhAYoodTIIIaoOOCSonyKaAsSdoACIaIiFIiMfUeJItaKEISiOuxDOWcRoiTYNLYTONRuaaIeinaaoIoysACRAuSyAypAoswKAayLvEaOtEEAXciHyiiaaayEFliEsgSaOiCAOEPYtDKOIGKiootHLdOzkiaaIPIIooaUaOUAIrAdAKlObEYiINleoOTEKSOTuTEeiaAEsiYUTiyIIaeROAsRmAAiIoiIgDylglMtAieBcihkoIrOieoIYuOouaKerYAOOiaMaIoht'
    translator=str.maketrans(strange,ascii_replacements)
    return input_text.translate(translator)
def removeAccentsFromAList(words):
  ls = []
  for word in words:
    word = removeAccents(word)
    ls.append(word)
  return ls


dictionary = {}
i = 0
#data is a a DataFrame with a single column, data[0] is a Series with all sentences 
for value in data[0]:
  dictionary[i] = removeAccents(value)
  i+=1
s = pd.Series(dictionary)

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices(s)
# tensorflow word2vec tutorial
# this is just changing a sentence to a sequence of numbers
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

text_dataset = dataset
max_features = 30000  # Maximum vocab size.
max_len = 2000  # Sequence length to pad the outputs to.
embedding_dims = 2

# Create the layer.
vectorize_layer = TextVectorization(
 max_tokens=max_features,
 output_mode='int',
 output_sequence_length=max_len)

# Now that the vocab layer has been created, call `adapt` on the text-only
# dataset to create the vocabulary. You don't have to batch, but for large
# datasets this means we're not keeping spare copies of the dataset.
vectorize_layer.adapt(text_dataset.batch(64))

# Create the model that uses the vectorize text layer
model = tf.keras.models.Sequential()

# Start by creating an explicit input layer. It needs to have a shape of
# (1,) (because we need to guarantee that there is exactly one string
# input per batch), and the dtype needs to be 'string'.
model.add(tf.keras.Input(shape=(1,), dtype=tf.string))

# The first layer in our model is the vectorization layer. After this
# layer, we have a tensor of shape (batch_size, max_len) containing vocab
# indices.
model.add(vectorize_layer)

# Now, the model can map strings to integers, and you can add an embedding
# layer to map these integers to learned embeddings.
input_data = [["Janusz"]]
model.predict(input_data)



array([[1227,    0,    0, ...,    0,    0,    0]])

In [ ]:
def vectorize_text(text):
  text = tf.expand_dims(text, -1)
  return tf.squeeze(vectorize_layer(text))
# Vectorize the data in text_ds.
text_vector_ds = dataset.batch(1024).prefetch(1).map(vectorize_layer).unbatch()

In [ ]:
sequences = list(text_vector_ds.as_numpy_iterator())

In [ ]:
#here we generate "tensorflow-friendly" training data, basing on skip-grams
BATCH_SIZE = 1024
BUFFER_SIZE = 10000
AUTOTUNE = tf.data.AUTOTUNE
# Generates skip-gram pairs with negative sampling for a list of sequences
# (int-encoded sentences) based on window size, number of negative samples
# and vocabulary size.
def generate_training_data(sequences, window_size, num_ns, vocab_size, seed):
  # Elements of each training example are appended to these lists.
  targets, contexts, labels = [], [], []

  # Build the sampling table for vocab_size tokens.
  sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(vocab_size)

  # Iterate over all sequences (sentences) in dataset.
  for sequence in tqdm.tqdm(sequences):

    # Generate positive skip-gram pairs for a sequence (sentence).
    positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
          sequence, 
          vocabulary_size=vocab_size,
          sampling_table=sampling_table,
          window_size=window_size,
          negative_samples=0)

    # Iterate over each positive skip-gram pair to produce training examples 
    # with positive context word and negative samples.
    for target_word, context_word in positive_skip_grams:
      context_class = tf.expand_dims(
          tf.constant([context_word], dtype="int64"), 1)
      negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
          true_classes=context_class,
          num_true=1, 
          num_sampled=num_ns, 
          unique=True, 
          range_max=vocab_size, 
          seed=42, 
          name="negative_sampling")

      # Build context and label vectors (for one target word)
      negative_sampling_candidates = tf.expand_dims(
          negative_sampling_candidates, 1)

      context = tf.concat([context_class, negative_sampling_candidates], 0)
      label = tf.constant([1] + [0]*num_ns, dtype="int64") #one skip-gram is correct, other's ain't

      # Append each element from the training example to global lists.
      targets.append(target_word)
      contexts.append(context)
      labels.append(label)

  return targets, contexts, labels

In [ ]:
targets, contexts, labels = generate_training_data(
    sequences=sequences, 
    window_size=3, 
    num_ns=4, 
    vocab_size=30000, 
    seed=42)
print(len(targets), len(contexts), len(labels))

100%|██████████| 39566/39566 [12:08<00:00, 54.31it/s]

1581464 1581464 1581464


In [ ]:
class Word2Vec(Model):
  def __init__(self, vocab_size, embedding_dim):
    super(Word2Vec, self).__init__()
    self.target_embedding = Embedding(vocab_size, 
                                      embedding_dim,
                                      input_length=1,
                                      name="w2v_embedding", )
    #^ this is the layer we've wanted all along
    self.context_embedding = Embedding(vocab_size, 
                                       embedding_dim, 
                                       input_length=4+1)
    self.dots = Dot(axes=(3,2))
    self.flatten = Flatten()

  def call(self, pair):
    target, context = pair
    we = self.target_embedding(target)
    ce = self.context_embedding(context)
    dots = self.dots([ce, we])
    return self.flatten(dots)

In [ ]:
def custom_loss(x_logit, y_true):
      return tf.nn.sigmoid_cross_entropy_with
      _logits(logits=x_logit, labels=y_true)

In [ ]:
embedding_dim = 128
word2vec = Word2Vec(30000, embedding_dim)
word2vec.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices(((targets, contexts), labels))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset = dataset.cache().prefetch(buffer_size=AUTOTUNE)


In [ ]:
word2vec.fit(dataset, epochs=5, callbacks=[tensorboard_callback]) #I usually use 30 epochs, but they take a while

Epoch 1/5
1544/1544 [==============================] - 100s 63ms/step - loss: 1.5558 - accuracy: 0.2969
Epoch 2/5
1544/1544 [==============================] - 93s 60ms/step - loss: 1.3987 - accuracy: 0.4254
Epoch 3/5
1544/1544 [==============================] - 93s 60ms/step - loss: 1.2499 - accuracy: 0.5223
Epoch 4/5
1544/1544 [==============================] - 94s 61ms/step - loss: 1.0897 - accuracy: 0.6104
Epoch 5/5
1544/1544 [==============================] - 93s 61ms/step - loss: 0.9325 - accuracy: 0.6849


In [ ]:
#get the weights and save later for further analysis
weights = word2vec.get_layer('w2v_embedding').get_weights()[0]
vocab = vectorize_layer.get_vocabulary()

In [ ]:
word2vec.get_layer('w2v_embedding')(vectorize_text("")[0]) #exact same embed for empty text - correct

<tf.Tensor: shape=(128,), dtype=float32, numpy=
array([-0.01960925,  0.00041066,  0.02948524,  0.04857682,  0.01769909,
       -0.02511126,  0.01209472, -0.03513237, -0.02793828,  0.01949591,
        0.01478063, -0.04959884,  0.02569914,  0.01586301, -0.03796757,
       -0.04093102, -0.0065673 , -0.01891701,  0.03462904,  0.03140365,
       -0.04794027, -0.01309193, -0.02947944,  0.03102602, -0.02248325,
        0.03804251,  0.00061284,  0.00643868,  0.00890405, -0.0391701 ,
        0.04105103, -0.04599273, -0.0041225 ,  0.01226106, -0.02604914,
       -0.02690941,  0.03431848,  0.04468158, -0.03061701,  0.00328814,
       -0.02319564, -0.0473825 , -0.00615723,  0.01181059,  0.04595621,
        0.036067  , -0.00806782, -0.03131787,  0.03535627,  0.02992752,
        0.01100625,  0.0384342 , -0.03894104, -0.02465555,  0.00917498,
       -0.01808133,  0.04533707,  0.03221381, -0.02424024,  0.00329977,
        0.02907927, -0.00425981,  0.03328582, -0.0398258 ,  0.04582288,
        0.015276

In [ ]:
weights[0] #most popular token is just empty one - correct

array([-0.01960925,  0.00041066,  0.02948524,  0.04857682,  0.01769909,
       -0.02511126,  0.01209472, -0.03513237, -0.02793828,  0.01949591,
        0.01478063, -0.04959884,  0.02569914,  0.01586301, -0.03796757,
       -0.04093102, -0.0065673 , -0.01891701,  0.03462904,  0.03140365,
       -0.04794027, -0.01309193, -0.02947944,  0.03102602, -0.02248325,
        0.03804251,  0.00061284,  0.00643868,  0.00890405, -0.0391701 ,
        0.04105103, -0.04599273, -0.0041225 ,  0.01226106, -0.02604914,
       -0.02690941,  0.03431848,  0.04468158, -0.03061701,  0.00328814,
       -0.02319564, -0.0473825 , -0.00615723,  0.01181059,  0.04595621,
        0.036067  , -0.00806782, -0.03131787,  0.03535627,  0.02992752,
        0.01100625,  0.0384342 , -0.03894104, -0.02465555,  0.00917498,
       -0.01808133,  0.04533707,  0.03221381, -0.02424024,  0.00329977,
        0.02907927, -0.00425981,  0.03328582, -0.0398258 ,  0.04582288,
        0.01527609, -0.04491785, -0.00968845,  0.02751957,  0.01

In [ ]:
# save as files
# I've commented these out after first run
# you can download these if you want to

with open('/content/Data/fullvectors.tsv', 'w', encoding='utf-8') as writefile:
  for index, word in enumerate(vocab):
    vec = weights[index]
    writefile.write('\t'.join([str(x) for x in vec]) + "\n")


with open('/content/Data/fullvocab.tsv', 'w', encoding='utf-8') as writefile:
  for index, word in enumerate(vocab):
    vec = weights[index]
    writefile.write(word + "\n")



In [ ]:
df = pd.DataFrame(index = vocab, data = weights)

In [ ]:
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127
,-0.019609,0.000411,0.029485,0.048577,0.017699,-0.025111,0.012095,-0.035132,-0.027938,0.019496,0.014781,-0.049599,2.569914e-02,0.015863,-0.037968,-0.040931,-0.006567,-0.018917,0.034629,0.031404,-0.047940,-0.013092,-0.029479,0.031026,-0.022483,0.038043,0.000613,0.006439,0.008904,-0.039170,0.041051,-0.045993,-0.004122,0.012261,-0.026049,-0.026909,0.034318,0.044682,-0.030617,0.003288,...,0.044762,0.040714,0.002698,0.017913,0.028170,-0.034683,-0.042108,-0.038484,-0.022408,-0.048250,0.020904,-0.035518,0.000594,0.016216,-0.042811,-0.008450,0.030189,0.041560,0.032490,0.037229,0.039701,0.036696,0.007075,0.023077,-0.035332,-0.038625,-0.040687,0.027334,0.046910,0.021500,-0.043208,0.026617,-0.008462,-0.002756,0.025424,0.027841,-0.033873,0.033363,0.041117,-0.024440
[UNK],0.148914,-0.192986,-0.189580,0.290793,-0.224534,0.042024,0.269316,0.146291,0.117027,0.074500,0.320065,0.120152,-3.536457e-02,-0.213201,-0.235622,-0.154314,-0.276462,-0.090629,-0.126104,-0.175772,-0.318695,-0.300283,0.352409,-0.118736,-0.143692,-0.091443,0.056670,0.085348,-0.002904,0.071841,-0.124421,-0.193727,0.189579,-0.187491,0.084501,0.028138,-0.144048,0.050795,0.068418,0.168266,...,0.097728,0.216255,0.222579,-0.103438,0.184119,0.217191,0.254816,0.001801,-0.154516,-0.042999,-0.057817,0.067807,-0.120988,-0.065118,0.348352,0.113530,0.130194,-0.076573,0.107864,-0.094355,0.175337,-0.242576,0.171619,0.014022,0.054651,0.241283,0.084248,-0.237860,-0.248604,-0.003279,0.101062,0.289045,-0.163941,-0.120817,-0.077792,-0.330287,-0.141973,0.023744,0.060056,0.308015
w,-0.491319,0.117181,-0.010057,0.252278,-0.140171,0.015566,0.330867,0.164992,-0.335459,0.067281,0.306156,0.145511,-2.697992e-01,-0.171528,0.126499,-0.452856,-0.125508,0.003373,0.253085,0.040927,-0.050383,-0.317377,0.030672,-0.027369,-0.267874,0.094675,0.239406,0.341355,-0.130057,-0.340373,-0.031923,-0.221872,-0.277674,-0.014338,-0.013118,-0.234002,-0.005961,0.044965,0.047759,-0.029994,...,0.109679,0.219228,-0.105698,0.057962,0.227038,-0.407336,-0.221217,-0.035060,0.059393,-0.317225,-0.059218,0.270338,0.285820,-0.131852,-0.267749,-0.318935,0.028312,-0.133861,-0.252877,-0.151453,0.164515,0.386089,-0.474654,-0.221890,-0.085740,-0.099473,-0.008746,-0.068403,-0.122934,0.155207,0.196459,-0.365627,-0.009289,-0.042122,0.041137,0.058899,-0.180035,-0.105883,0.279341,0.294245
i,-0.244589,-0.225947,0.026199,0.063732,-0.322831,0.227737,-0.155107,-0.197771,-0.187987,-0.075027,-0.198954,0.037597,-5.897236e-02,-0.045517,-0.011706,-0.388227,-0.477904,0.008508,0.024719,0.076449,-0.161913,-0.694998,0.036837,0.231759,-0.125236,0.287605,-0.070243,-0.091724,-0.156528,-0.048758,0.186064,-0.058096,-0.388032,0.185053,-0.335603,0.093245,0.103375,-0.206494,-0.615458,-0.016482,...,-0.171466,-0.177653,-0.186646,-0.097905,0.078559,0.353036,0.094215,-0.065443,-0.032058,-0.078489,0.220587,-0.101265,0.344948,-0.082424,0.078161,0.055689,-0.048073,0.127868,0.175778,0.094689,0.307801,0.137502,-0.061490,0.115698,0.118691,0.301369,-0.202587,-0.102831,0.093794,0.103956,-0.101978,-0.199827,0.249766,0.193076,0.203371,-0.029429,0.217458,0.210438,-0.101933,0.303370
sie,-0.168541,-0.163091,0.238549,-0.142499,0.055736,0.020237,0.226484,-0.127130,-0.006252,0.459998,-0.360074,0.248754,-1.723825e-01,-0.398312,0.199874,-0.008327,-0.047690,0.165809,0.253640,-0.222853,-0.096644,0.062145,0.179820,0.044522,0.032161,-0.303667,-0.019165,-0.120950,-0.001055,0.044815,0.156529,0.154988,-0.188910,0.066396,0.077228,0.052609,0.011411,0.207014,-0.183731,-0.089316,...,0.153625,0.219437,-0.047230,-0.052246,-0.293211,0.122125,0.213851,0.077447,0.322474,-0.155164,0.046152,0.173494,-0.286430,0.134175,-0.134256,-0.357407,-0.199585,0.219326,-0.130081,0.040471,0.227247,-0.209316,0.152119,0.346511,0.227165,-0.104914,0.256618,-0.1382

In [ ]:
df.index = vocab
df.index

Index(['', '[UNK]', 'w', 'i', 'sie', 'na', 'nie', 'z', 'do', 'to',
       ...
       'tuli', 'tula', 'tudziez', 'tucholskiego”', 'tt', 'trzymamy',
       'trzymacie', 'trzydziestej', 'trzydniowa', 'trzezwym'],
      dtype='object', length=30000)

In [ ]:

def similarity(text1, text2):
  try:
    return cosine_similarity(df.loc[text1].values.reshape(1,-1), df.loc[text2].values.reshape(1,-1))[0][0]
  except KeyError:
    return np.nan
    pass
def generate_scoreboard(names, objects):
  table = []
  for name in names:
    row = []
    for obj in objects:
      row.append(similarity(name, obj))
    table.append(row)
  df = pd.DataFrame(data = table, index = names, columns = objects)
  df.dropna(how = 'all', axis = 1, inplace=True)
  df.dropna(how = 'all', axis = 0, inplace=True)
  return df

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity 
positive_adjectives = removeAccentsFromAList(positive_adjectives)
positive_words = removeAccentsFromAList(positive_words)
negative_adjectives = removeAccentsFromAList(negative_adjectives)
negative_words = removeAccentsFromAList(negative_words)
first_names = removeAccentsFromAList(first_names)
all_names = removeAccentsFromAList(all_names)

In [ ]:
#let's see how closely related are names and adjectives and other emotionally charged words 
positive_scores_from_adj = generate_scoreboard(all_names, positive_adjectives)
positive_scores_from_words = generate_scoreboard(all_names, positive_words)
negative_scores_from_words = generate_scoreboard(all_names, negative_words)
negative_scores_from_adj = generate_scoreboard(all_names, negative_adjectives)

In [ ]:
positive_scores_from_adj

,swietny,ladny,sloneczny,wierny,pozytywny,szczesliwy,zdrowy,mily,czysty,udany,wygodny,bezpieczny,zadowolony,inteligentny,wiosenny,wesoly,genialny,spokojny,sliczny,kochany,radosny,cieply,kolorowy,wspanialy
adam,0.209953,0.019502,0.116660,0.200267,0.019018,0.150206,0.084353,0.132001,0.049765,0.073586,-0.021074,0.015149,0.308417,-0.039445,0.208873,0.114774,0.308689,0.006136,0.169266,0.225358,0.341939,0.078214,0.125708,0.242705
adolf,0.014604,-0.049959,0.331649,0.007304,-0.001427,0.112594,0.180637,-0.013878,-0.007623,0.008055,-0.058655,0.062576,0.042317,0.048551,0.196243,0.043549,0.137294,-0.003225,0.144975,0.106260,0.119570,0.080573,0.290898,-0.025883
adrian,0.133736,-0.176086,0.057103,0.115663,-0.039265,-0.149858,-0.082814,-0.005673,0.150521,-0.159258,-0.096467,0.030144,-0.060509,0.035377,0.125671,-0.059051,0.026638,-0.103714,0.024016,-0.004850,0.126007,0.226781,0.356719,-0.061093
agata,0.272467,0.372958,0.163537,0.339554,0.148737,0.016835,0.247479,0.243003,0.237842,0.176996,0.180141,0.132773,0.138258,0.412856,0.070659,0.141780,0.065465,-0.102048,0.250413,0.333777,0.236681,0.285648,0.273163,0.330033
ala,0.112111,0.286430,-0.016511,0.148867,-0.033296,0.019752,0.132273,0.126338,0.017200,0.163535,-0.138741,-0.009173,0.015229,0.104295,-0.066193,0.253672,-0.143039,-0.116406,-0.031427,0.233180,-0.012403,0.062151,0.007985,0.023175
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
wlodzimierza,0.141501,-0.040933,0.258151,0.224312,0.260332,0.026714,0.044860,0.001093,0.316790,0.034354,0.014355,-0.097991,-0.018117,0.224918,0.215481,0.045212,0.156072,-0.137731,0.111827,0.083859,0.239973,0.127151,0.222745,0.226185
zbigniewa,0.083233,-0.129208,0.047808,0.102066,0.074489,-0.083297,0.022389,-0.020012,0.008438,-0.107192,0.090142,-0.050944,-0.012059,0.049328,0.398024,0.118945,0.311617,-0.042357,0.287238,-0.020079,0.221876,0.137663,0.200477,0.178533
zbyszek,0.161460,0.333473,0.179579,0.222191,0.205030,0.231409,0.210864,0.305914,0.101198,0.273924,0.109265,-0.021581,0.420746,0.254698,0.021671,0.140429,0.086384,0.078774,0.241691,0.320183,0.050125,-0.030877,0.023723,0.208985
zdzislawa,0.057241,-0.226562,-0.025422,0.049150,0.065444,-0.075575,0.127453,-0.010636,-0.001445,-0.053015,-0.108218,0.024259,0.154004,0.002669,0.275731,-0.038664,0.381208,0.028129,0.148193,0.069742,0.219236,0.039161,0.186163,-0.097312


In [ ]:
# let's try to find "strongest" positive and negative features of a name, basing on encodings
# in other words - what would be predicted features of a baby with a given name according to the language itself?

def rank_features_of_a_name(name, df):
  return df.sort_values(by = name, axis = 1).loc[name]

rank_features_of_a_name("harry", positive_scores_from_adj).tail(10)
#harry scores good grades in genius, great and good-hearted, looks like the books are reflected here well
#this snippet requires using all_names to generate scoreboards, Harry isn't a traditional Polish name

genialny        0.284081
inteligentny    0.298365
mily            0.325473
czysty          0.347986
cieply          0.359100
swietny         0.365574
kochany         0.369772
kolorowy        0.374882
radosny         0.395234
sliczny         0.471051
Name: harry, dtype: float64

In [ ]:
#Good sanity check - look up Adolf in the context of negative adjectives, if any name had strong associations it would be that
#there might have been a couple more Adolfs during the decades, but let's be honest - you know the one responsible for 95%+ of articles
#this snippet requires using all_names to generate scoreboards, Adolf isn't exactly popular anymore
rank_features_of_a_name("adolf", negative_scores_from_adj).tail(10)
#some of the related are vain, mad, spendy, offended, depressed, despairing, pedantic, stubborn, arrogant
#sounds about right

niespokojny    0.091469
ponury         0.121293
tajny          0.141180
olbrzymi       0.175186
uparty         0.195770
daleki         0.196761
brudny         0.208768
zrozpaczony    0.214191
glosny         0.237276
policyjny      0.255919
Name: adolf, dtype: float64

In [ ]:
#Okay, what would be his strong points then?
rank_features_of_a_name("adolf", positive_scores_from_adj).tail(10)
#surprisingly the top word appears to be "kind hearted", perhaps in context of his closest environment?
#also genius, trusted, loyal, intelligent, lovely, cheerful
#taking into account that he was highly bipolar this might be sensible

cieply        0.080573
kochany       0.106260
szczesliwy    0.112594
radosny       0.119570
genialny      0.137294
sliczny       0.144975
zdrowy        0.180637
wiosenny      0.196243
kolorowy      0.290898
sloneczny     0.331649
Name: adolf, dtype: float64

In [ ]:
for name in all_names:
  print(name) #list of names for reference

Jakub
Julia
Kacper
Wiktoria
Zuzanna
Szymon
Natalia
Aleksandra
Mateusz
Maja
Michal
Oliwia
Filip
Lena
Amelia
Jan
Bartosz
Zofia
Dawid
Piotr
Mikolaj
Antoni
Adam
Weronika
Hanna
Martyna
Wojciech
Alicja
Wiktor
Anna
Aleksander
Karolina
Patryk
Emilia
Maciej
Kamil
Maria
Nikola
Dominik
Igor
Franciszek
Pawel
Gabriela
Marcel
Magdalena
Oskar
Patrycja
Karol
Klaudia
Krzysztof
Paulina
Tomasz
Maksymilian
Kinga
Hubert
Adrian
Oliwier
Bartlomiej
Katarzyna
Laura
Milosz
Sebastian
Stanislaw
Nadia
Dominika
Alan
Milena
Nikodem
Lukasz
Agata
Antonina
Krystian
Marta
Damian
Marcin
Daniel
Kornelia
Pola
Iga
Malgorzata
Joanna
Michalina
Leon
Konrad
Liliana
Gabriel
Tymoteusz
Fabian
Marcelina
Jagoda
Kamila
Tymon
Daria
Ignacy
Sandra
Julian
Izabela
Barbara
Nina
Rafal
Helena
Przemyslaw
Grzegorz
Artur
Roksana
Eryk
Agnieszka
Ksawery
Justyna
Natan
Kaja
Sara
Ewa
Blazej
Monika
Blanka
Lucja
Radoslaw
Olga
Olaf
Anastazja
Borys
Kuba
Klara
Kajetan
Robert
Marek
Angelika
Adrianna
Arkadiusz
Malwina
Norbert
Cezary
Matylda
Eliza
Urszula
G

In [ ]:
user_name = "mateusz" #@param {type:"string"}
user_df = positive_scores_from_adj #@param ["positive_scores_from_adj", "negative_scores_from_adj", "positive_scores_from_words", "negative_scores_from_words"] {type:"raw"}
#Try it yourself!
#Type in a name form field and press Shift+Enter to automatically execute this cell and find positive/negative adjectives/words closest to that name
rank_features_of_a_name(user_name, user_df).tail(10)

wspanialy     0.142205
czysty        0.161894
zadowolony    0.165756
wierny        0.198330
sloneczny     0.202845
kochany       0.243057
radosny       0.253426
zdrowy        0.272983
mily          0.292532
sliczny       0.335744
Name: mateusz, dtype: float64

In [ ]:
user_word = "swietny" #@param {type:"string"}
user_df = positive_scores_from_adj #@param ["positive_scores_from_adj", "negative_scores_from_adj", "positive_scores_from_words", "negative_scores_from_adj"] {type:"raw"}
user_names = pop_names #@param ["all_names", "pop_names", "pop_boy_names", "pop_girl_names"] {type:"raw"}
def rank_names_by_word(word, df, names=user_names):
  return df.loc[list(set(names).intersection(set(df.index)))].sort_values(by = word)[[word]]

rank_names_by_word(user_word, user_df).tail(10) #what names are the closest to given adjective (happy by default)?

,swietny


In [ ]:
user_df = positive_scores_from_adj #@param ["positive_scores_from_adj", "positive_scores_from_words", "negative_scores_from_adj", "negative_scores_from_words"] {type:"raw"}
user_positive = True #@param {type:"boolean"}
user_names = pop_boy_names #@param ["all_names", "pop_names", "pop_boy_names", "pop_girl_names"] {type:"raw"}

#is there a "best name"?
#let's find out!
#note that you can change user_names to boy or girl names if that's interesting for you

def rank_names(df, positive=True, names=pop_names):
  if positive == True:
    return df.loc[list(set(names).intersection(set(df.index)))].mean(axis = 1, skipna=True).sort_values()
  else:
    return (df.loc[list(set(names).intersection(set(df.index)))].mean(axis = 1, skipna=True)*- 1.).sort_values()


rank_names(user_df, user_positive, user_names).tail(50) 
#try it yourself, leaving user_positive unchecked flips the values multiplying them by -1
#scoring negative dataframes this way ensures the "least bad names" will make their way to the tail (the top)
#rank_names gives list of names that "are best" in given context, the higher score the better

Series([], dtype: float64)

In [ ]:
overall_scores_words = pd.concat((positive_scores_from_words * 1/((len(positive_scores_from_words.columns)/len(negative_scores_from_words.columns))), negative_scores_from_words* -1.), axis =1) 
# because there is many more negative words than positive ones, I'm scaling positive scores by 1/(number of positive columns/number of negative columns)
overall_scores_adj = pd.concat((positive_scores_from_adj * (len(negative_scores_from_adj.columns)/len(positive_scores_from_adj.columns)), negative_scores_from_adj * -1), axis=1)

In [ ]:
rank_names(overall_scores_adj).tail(30)

#"best balanced", "with least powerful downside" names appear to be about 80% female names, overall "non-traditional" names
#conversly, changing "names" parameter of the function to all_names yields about 30% of "traditional" names


Series([], dtype: float64)

In [ ]:
def rank_features_of_a_name(name, df):
  return df.sort_values(by = name, axis = 1).loc[name]

rank_features_of_a_name("adolf", negative_scores_from_words).tail(40)

kosa           0.193556
uparty         0.195770
daleki         0.196761
sad            0.200885
cmentarz       0.203671
brudny         0.208768
drzec          0.210855
zrozpaczony    0.214191
test           0.216537
protestowac    0.217099
amunicja       0.222160
lawina         0.222233
dol            0.223222
armia          0.226181
spadac         0.227062
awaria         0.229283
lza            0.230783
odpady         0.233680
glosny         0.237276
rektor         0.246231
lancuch        0.248075
policyjny      0.255919
kosztowac      0.257989
pila           0.267794
benzyna        0.270776
krawedz        0.273396
ograniczac     0.277806
scieki         0.279038
pelnomocnik    0.292458
kopalnia       0.298993
kurator        0.317358
studnia        0.321180
pozegnanie     0.324402
bariera        0.339871
mikrofon       0.344607
radar          0.349829
tunel          0.395297
agent          0.422008
rakieta        0.454472
szczecina      0.471121
Name: adolf, dtype: float64

In [ ]:
def rank_names_by_word(word, df):
  return df.sort_values(by = word)[[word]]

rank_names_by_word("inteligentny", positive_scores_from_adj)

,inteligentny
emila,-0.240271
boguslawa,-0.205497
wieslawa,-0.202524
donald,-0.156861
daniela,-0.146317
...,...
kinga,0.392913
dawid,0.393568
agata,0.412856
klaudia,0.415688


In [ ]:
def rank_names(df, positive=True):
  if positive == True:
    return df.mean(axis = 1, skipna=True).sort_values()
  else:
    return (df.mean(axis = 1, skipna=True)*- 1.).sort_values()

rank_names(positive_scores_from_adj).tail(50) #50 names that score the highest on average on positive adjectives

krystian     0.202126
karolina     0.202695
wiktoria     0.203179
patrycja     0.203631
waclaw       0.203802
jagienka     0.203959
kamil        0.204431
feliks       0.204719
teofil       0.206882
agata        0.207042
kinga        0.208385
august       0.209009
magda        0.209186
dominik      0.209306
mariola      0.209389
bogumil      0.209491
wanda        0.209823
tatiana      0.210960
alfred       0.212717
klara        0.214094
kasia        0.216022
nina         0.216488
igor         0.217371
alojzy       0.217503
roland       0.222718
bartosz      0.223661
aniela       0.223894
leonarda     0.225533
natalia      0.226951
laura        0.228351
alfreda      0.228744
irena        0.230481
harry        0.234016
ewelina      0.235522
jurek        0.237369
zygmunt      0.238174
eugeniusz    0.239597
milena       0.243529
klaudia      0.249645
roza         0.253636
roza         0.253636
roza         0.253636
daniel       0.254463
paulina      0.254929
walentyna    0.259247
karol     

In [ ]:
rank_features_of_a_name("mateusz", positive_scores_from_words).tail(40)

talent         0.104615
sierpien       0.109684
weekend        0.110269
spac           0.111221
spelnienie     0.114530
medal          0.115683
wiosenny       0.115789
sport          0.117175
dawac          0.122449
zwyciezca      0.122496
udany          0.126811
cud            0.133839
toast          0.133921
genialny       0.141914
oklaski        0.141993
wspanialy      0.142205
deser          0.142614
ulga           0.143260
hobby          0.148756
super          0.155953
czysty         0.161894
przyjemnosc    0.163100
zapas          0.165151
zadowolony     0.165756
przyjaciel     0.181482
prezent        0.183031
wierny         0.198330
sloneczny      0.202845
posilek        0.204864
muzyka         0.208095
tort           0.213998
pomysl         0.225089
poduszka       0.239559
kochany        0.243057
maj            0.245243
radosny        0.253426
zart           0.268225
zdrowy         0.272983
mily           0.292532
sliczny        0.335744
Name: mateusz, dtype: float64

In [ ]:
full_scores_words = positive_scores_from_words.merge(negative_scores_from_words*-1.0, left_index=True, right_index=True)
full_scores_words = full_scores_words[~full_scores_words.index.duplicated(keep='first')]
rank_names(full_scores_words).tail(40) #40 "on average" best names, or maybe with the smallest downside?

wanda       -0.011335
sylwester   -0.011155
kuba        -0.010576
blanka      -0.010516
adolfa      -0.010478
wladimir    -0.010042
joachim     -0.009217
magda       -0.008700
dominika    -0.008650
boza        -0.008280
dominik     -0.007897
rafala      -0.007705
maja        -0.007578
mateusz     -0.007483
izabela     -0.006241
klaudia     -0.005555
irena       -0.004979
zenon       -0.004627
slawa       -0.004457
iwona       -0.003555
karolina    -0.001935
nina        -0.001490
kacper      -0.001118
roland      -0.001097
hanka       -0.000870
natalia     -0.000800
krzysiek     0.000015
marianna     0.001347
ala          0.004722
emanuel      0.005168
leslaw       0.005232
oktawia      0.006575
pawla        0.007399
renata       0.007474
basia        0.007884
ola          0.010818
wiktoria     0.011504
lucja        0.014017
ela          0.016777
bartek       0.018164
dtype: float64

In [ ]:
df[:10]

In [ ]:
full_scores_adj = positive_scores_from_adj.merge(negative_scores_from_adj*-1.0, left_index=True, right_index=True)
full_scores_adj = full_scores_adj[~full_scores_adj.index.duplicated(keep='first')] #reduce to adjectives only
rank_names(full_scores_adj).tail(40)

bronislaw    0.021522
magdalena    0.021988
czeslaw      0.022067
adolfa       0.022360
rafala       0.022376
miroslawa    0.022846
alfred       0.023110
zdzislaw     0.023271
waldemar     0.023438
dariusz      0.023664
iwona        0.023718
albert       0.024306
wieslaw      0.025136
mariusz      0.025452
marianna     0.025877
alojzy       0.026066
tomasz       0.026154
maciej       0.026416
wladyslaw    0.026678
bogumila     0.026990
eugeniusz    0.027207
bartosz      0.027264
miroslaw     0.027364
daniel       0.027379
zenona       0.027395
magda        0.028174
marian       0.028367
joachim      0.029553
zbigniewa    0.030879
peter        0.031519
klaudia      0.031789
izabela      0.032572
wiktoria     0.033645
mateusz      0.034745
lucja        0.036149
walter       0.037312
irena        0.038604
wladimir     0.039713
roland       0.042168
zenon        0.043129
dtype: float64